In [6]:
import os
import youtube_dl
import numpy as np
import subprocess
import urllib
import urllib2
from bs4 import BeautifulSoup
import pyvtt
import wave
import indicoio
import librosa

indicoio.config.api_key = <your api key>

In [7]:
def download_video(url, name, language='en'):
    """Download soundtrack and subtitles. If there's no subtitles, raise error (better - raise it ASAP)."""
    #"source activate jupyterhub_py3 && 
    code = os.system("youtube-dl {0} --write-sub --sub-lang {1} --output {2}.mp4".format(url, language, name))
    assert code==0, "Something wrong"
    if not os.path.exists(name+".en.vtt"):
        os.system("rm -rf ./"+name+".mp4")
        raise ValueError("No subtitles!")
    
    return name+".mp4", name+".en.vtt"

In [8]:
def mp4_to_wav(name):
    if not name.endswith(".mp4"): name+=".mp4"
    os.system("avconv -i ./{0} -vn -f wav {1}.wav -ac 1".format(name, name[:-4]))
    os.system("rm -rf ./"+name)
    return "{0}.wav".format(name[:-4])

In [9]:
def auto_download(urls, language_of_subs="en"):
    sounds=[]
    subs = []
    for n, url in enumerate(urls):
        name="video"+str(n)
        try:
            name, sub = download_video(url, name, language_of_subs)
        except:
            print("Video \"{0}\" doesn`t have subtitle so it is missed".format(name))
        else:
            sound_name = mp4_to_wav(name)
            sounds.append(sound_name)
            subs.append(sub)
    return sounds, subs

In [10]:
def restore(mun_eatems):
    sounds=[]
    subs = []
    for num in range(mun_eatems):
        sound_name="video"+str(num)+".wav"
        sub_names="video"+str(num)+".en.vtt"
        sounds.append(sound_name)
        subs.append(subs)
    return sounds, subs

In [11]:
types = {
    1: 'int8',
    2: 'int16',
    4: 'int32'
}
type_to_width = {t:w for w,t in types.items()}

def slice_wav(wav_name, subt_name):
    wav = wave.open(wav_name, mode="r")
    (nchannels, sampwidth, framerate, nframes, comptype, compname) = wav.getparams()
    duration = float(nframes) / framerate


    def time_to_frame_ix(time):
        return int(round(time.ordinal/1000.*framerate))

    def get_line_from_file(line):
        wav.setpos(time_to_frame_ix(line.start))
        raw_data = wav.readframes( (time_to_frame_ix(line.end) - time_to_frame_ix(line.start))*nchannels)
        series = np.fromstring(raw_data, dtype=types[sampwidth]).reshape([-1,nchannels]).mean(-1).astype(types[sampwidth])
        return series
    
    return [(get_line_from_file(line),line.text) for line in pyvtt.WebVTTFile.open(subt_name)], framerate

In [12]:
def make_url_list(chanel_url):
    #query = urllib.quote(textToSearch)
    #url = "https://www.youtube.com/results?search_query=" + 'Dogs'  #for search
    response = urllib2.urlopen(chanel_url)
    html = response.read()
    soup = BeautifulSoup(html)
    urls=['https://www.youtube.com' + vid['href'] for vid in soup.findAll(attrs={'class':'yt-uix-tile-link'})]
    return urls

In [13]:
def get_sentiment(wav_name, subt_name):
    pairs,fr = slice_wav(wav_name, subt_name)
    sentiment = indicoio.sentiment([p[1] for p in pairs])
    return sentiment

In [14]:
norm_len = 1007

def sound_normalization(wav_name, subt_name):
    pairs, framerate = slice_wav(wav_name, subt_name)
    sound = [librosa.logamplitude(librosa.feature.melspectrogram(p[0], sr=framerate, n_mels=128), ref_power=np.max)  for p in pairs]
    lens = [wave.shape[1] for wave in sound]
    if norm_len == 0:
        norm_len = int(np.percentile(lens, q = 90))
    
    norm_sound = np.zeros([len(sound), 128, norm_len], dtype='float32')
    
    global norm_len
    for i,s in enumerate(sound):
        if s.shape[1]>=norm_len:
            norm_sound[i, :, :]= np.array(s[:, :norm_len])
        else:
            #sound_wave = list(s)
            #while len(sound_wave) < norm_len: sound_wave.append(0)
            norm_sound[i, :, :s.shape[1]] = np.array(s)
    return norm_sound

<ipython-input-14-12ede8afe5e3>:12: SyntaxWarning: name 'norm_len' is assigned to before global declaration
  global norm_len


In [15]:
sounds, subtitles = auto_download(make_url_list("https://www.youtube.com/user/CGPGrey/videos"))

/root/miniconda/envs/rep_py2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /root/miniconda/envs/rep_py2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html.parser")

  markup_type=markup_type))


In [16]:
X = [np.array(sound_normalization(sounds[i], subtitles[i])) for i in range(len(sounds))]

In [17]:
#X = np.array(X)
X = np.concatenate(X,axis=0).reshape(-1, 128*1007)

In [18]:
y = [get_sentiment(sounds[i], subtitles[i]) for i in range(len(sounds))]

In [19]:
k=[]
[[k.append(j) for j in i] for i in y]
y = np.array(k)

In [ ]:
(X.shape, y.shape)

((2514, 128896), (2514,))

In [ ]:
import pickle
with open("X_dump", "w") as f:
    pickle.dump(X, f)
with open("y_dump", "w") as f:
    pickle.dump(y, f)

In [ ]:
import pickle
X = pickle.load(open("X_dump"))
y = pickle.load(open("y_dump"))

In [ ]:
import xgboost as xgb

In [ ]:
X_train, X_valid, y_train, y_valid = X[:2000], X[2000:], y[:2000], y[2000:]

In [ ]:
d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)

In [ ]:
params = {"objective":"reg:logistic", "eval_metric":"logloss", "n_estimators":10, "max_depth":3, "eta":0.02}
watchlist = [(d_train, "train"), (d_valid, "valid")]
model = xgb.train(params, d_train, 20, watchlist)